In [1]:
import pandas as pd

# 1. 타자와 투수에서 겹치는 인원들을 제거하여 독립적인 데이터셋 구성
 - 타자의 경우, 타석에 1회이상 들어선 인원만 우선 필터링
 - 타자와 투수에 모두 기록이 있는 선수가 있다면 경기수가 많은 포지션에 이를 할당, 적은 포지션에서는 이를 제거 (선수 판별기준: ID-선수명)

In [2]:
YEARS = list(range(2020, 2001, -1))
TEAMS = ['KIA', 'KT', 'LG', 'NC', 'SK', '넥센', '두산', '롯데', '삼성', '우리', '키움', '한화', '현대', '히어로즈']

for YEAR in YEARS:
    
    for TEAM in TEAMS:
        
        # 모든년도에 동일한 팀이 있는 것이 아니기에 오류발생시 로직구현
        try:
            hitter = pd.read_csv(f'선수데이터/타자/{TEAM}/타자정보_{TEAM}({YEAR}).csv')
        except:
            print(f'{TEAM}의 {YEAR}년도 데이터는 존재하지 않습니다.')
            continue
            
        # 타자데이터 불러온 후 간단한 전처리
        hitter['연도'] = YEAR
        hitter = hitter[['ID', '선수명', '팀명', '연도', '타율', '경기', '타석', '타수', '득점', '안타', '2루타', '3루타',
               '홈런', '루타', '타점', '희생번트', '희생플라이', '볼넷', '고의사구', '사구', '삼진', '병살타',
               '장타율', '출루율', 'OPS', '멀티히트', '득점권타율', '대타타율', '장타', '땅볼', '뜬공', '땅볼/뜬공',
               '결승타', '볼넷/삼진', '투구수/타석', '순수장타율', '추정득점', 'GPA']]
        hitter['key'] = hitter['ID'].astype('str') + '-' + hitter['선수명']
        hitter = hitter[hitter['타석'] > 0].reset_index(drop=True)

        
        # 투수데이터 불러온 후 간단한 전처리
        pitcher = pd.read_csv(f'선수데이터/투수/{TEAM}/투수정보_{TEAM}({YEAR}).csv')
        pitcher['연도'] = YEAR
        pitcher = pitcher[['ID', '선수명', '팀명', '연도', '평균자책점', '경기', '승리', '패배', '세이브', '홀드', '승률', '이닝',
               '피안타', '홈런', '볼넷', '사구', '삼진', '실점', '자책점', '이닝당_출루허용률', '완투', '완봉',
               '퀄리티스타트', '블론세이브', '타자수', '투구수', '피안타율', '2루타', '3루타', '희생번트', '희생플라이',
               '고의4구', '폭투', '보크', '선발', '선발승', '구원승', '종료', '세이브기회', '터프세이브', '병살타',
               '땅볼', '뜬공', '땅볼/뜬공', '인플레이타구타율', '투구수/경기', '투구수/이닝', '9이닝당 삼진',
               '9이닝당 볼넷', '삼진/볼넷', '피출루율', '피장타율', '피OPS']]
        pitcher['key'] = pitcher['ID'].astype('str') + '-' + pitcher['선수명']
        
        
        # 투수와 타자에 모두 있는 항목들을 확인
        dupli_keys = list(set(hitter['key']).intersection(set(pitcher['key'])))
        print(f'{TEAM}({YEAR})에서 타자와 투수에 중복된 선수들: {dupli_keys}')

        for dupli_key in dupli_keys:

            games_as_pitcher = int(pitcher.loc[pitcher['key'] == dupli_key, '경기'])
            games_as_hitter = int(hitter.loc[hitter['key'] == dupli_key, '경기'])

            if games_as_pitcher > games_as_hitter:
                hitter.drop(hitter[hitter['key'] == dupli_key].index, inplace=True)

            elif games_as_hitter > games_as_pitcher:
                pitcher.drop(pitcher[pitcher['key'] == dupli_key].index, inplace=True)

            elif games_as_hitter == games_as_pitcher:
                print('== 선수명: {dupli_key} == 타자와 투수의 경기수가 동일합니다. 확인이 필요합니다. ==')
                
                
        # key변수는 제거, 인덱스 초기화, 저장
        hitter = hitter.drop('key', axis=1).reset_index(drop=True)
        pitcher = pitcher.drop('key', axis=1).reset_index(drop=True)

        if (YEAR == 2020) & (TEAM == 'KIA'):
            hitter_final = hitter.copy()
            pitcher_final = pitcher.copy()

        else:
            hitter_final = pd.concat([hitter_final, hitter]).reset_index(drop=True)
            pitcher_final = pd.concat([pitcher_final, pitcher]).reset_index(drop=True)

KIA(2020)에서 타자와 투수에 중복된 선수들: ['62908-황윤호']
KT(2020)에서 타자와 투수에 중복된 선수들: []
LG(2020)에서 타자와 투수에 중복된 선수들: []
NC(2020)에서 타자와 투수에 중복된 선수들: []
SK(2020)에서 타자와 투수에 중복된 선수들: []
넥센의 2020년도 데이터는 존재하지 않습니다.
두산(2020)에서 타자와 투수에 중복된 선수들: []
롯데(2020)에서 타자와 투수에 중복된 선수들: []
삼성(2020)에서 타자와 투수에 중복된 선수들: ['69439-라이블리']
우리의 2020년도 데이터는 존재하지 않습니다.
키움(2020)에서 타자와 투수에 중복된 선수들: []
한화(2020)에서 타자와 투수에 중복된 선수들: ['68701-김진욱', '69737-노시환']
현대의 2020년도 데이터는 존재하지 않습니다.
히어로즈의 2020년도 데이터는 존재하지 않습니다.
KIA(2019)에서 타자와 투수에 중복된 선수들: ['65643-문경찬']
KT(2019)에서 타자와 투수에 중복된 선수들: ['65062-김재윤', '68050-강백호']
LG(2019)에서 타자와 투수에 중복된 선수들: []
NC(2019)에서 타자와 투수에 중복된 선수들: []
SK(2019)에서 타자와 투수에 중복된 선수들: []
넥센의 2019년도 데이터는 존재하지 않습니다.
두산(2019)에서 타자와 투수에 중복된 선수들: []
롯데(2019)에서 타자와 투수에 중복된 선수들: ['68556-정성종']
삼성(2019)에서 타자와 투수에 중복된 선수들: []
우리의 2019년도 데이터는 존재하지 않습니다.
키움(2019)에서 타자와 투수에 중복된 선수들: []
한화(2019)에서 타자와 투수에 중복된 선수들: []
현대의 2019년도 데이터는 존재하지 않습니다.
히어로즈의 2019년도 데이터는 존재하지 않습니다.
KIA(2018)에서 타자와 투수에 중복된 선수들: ['65618-황인준', '66643-헥터', '67640-유승철

롯데(2003)에서 타자와 투수에 중복된 선수들: []
삼성(2003)에서 타자와 투수에 중복된 선수들: []
우리의 2003년도 데이터는 존재하지 않습니다.
키움의 2003년도 데이터는 존재하지 않습니다.
한화(2003)에서 타자와 투수에 중복된 선수들: ['72756-피코타', '72749-마정길']
현대(2003)에서 타자와 투수에 중복된 선수들: []
히어로즈의 2003년도 데이터는 존재하지 않습니다.
KIA(2002)에서 타자와 투수에 중복된 선수들: []
KT의 2002년도 데이터는 존재하지 않습니다.
LG(2002)에서 타자와 투수에 중복된 선수들: ['93147-이상훈', '70121-경헌호']
NC의 2002년도 데이터는 존재하지 않습니다.
SK(2002)에서 타자와 투수에 중복된 선수들: []
넥센의 2002년도 데이터는 존재하지 않습니다.
두산(2002)에서 타자와 투수에 중복된 선수들: []
롯데(2002)에서 타자와 투수에 중복된 선수들: ['70444-이명호']
삼성(2002)에서 타자와 투수에 중복된 선수들: []
우리의 2002년도 데이터는 존재하지 않습니다.
키움의 2002년도 데이터는 존재하지 않습니다.
한화(2002)에서 타자와 투수에 중복된 선수들: []
현대(2002)에서 타자와 투수에 중복된 선수들: []
히어로즈의 2002년도 데이터는 존재하지 않습니다.


# 2. 팀명을 일괄 수정
 - 히어로즈, 넥센, 키움 -> 히어로즈/넥센/키움

In [3]:
def change_teamname2(x):  # KBO데이터
    
    if (x == '히어로즈') or (x == '넥센') or (x == '키움') or (x == '우리'):
        x = '우리/히어로즈/넥센/키움'
        
    return x

In [4]:
hitter_final['팀명'] = hitter_final['팀명'].apply(lambda x: change_teamname2(x))
pitcher_final['팀명'] = pitcher_final['팀명'].apply(lambda x: change_teamname2(x))

# 3. 데이터 내 외국인 제거

In [5]:
df = pd.read_csv('외인정보데이터/외인조사(수정).csv')
df

,연도,팀명,선수명
0,2002,KIA,다니엘 리오스
1,2002,KIA,마크 키퍼
2,2002,KIA,워렌 뉴선
3,2002,KIA,루디 펨버튼
4,2003,KIA,다니엘 리오스
...,...,...,...
523,2019,KT,윌리엄 쿠에바스
524,2019,KT,멜 로하스 주니어
525,2020,KT,윌리엄 쿠에바스
526,2020,KT,오드리사머 데스파이네


In [6]:
YEARS = list(range(2020, 2001, -1))
TEAMS = ['한화', 'KIA', 'LG', 'SK', '롯데', '삼성', '두산', '우리/히어로즈/넥센/키움', 'NC', 'KT', '현대']

for YEAR in YEARS:
    print('')
    print(f'{YEAR} 시작!!')
    
    for TEAM in TEAMS:
        print('')
        
        # 외인정보(수정).csv에서 연도/팀별 외국인명 추출
        names = []
        foreigners = df.loc[(df['연도'] == YEAR) & (df['팀명'] == TEAM)]
        for foreigner in foreigners['선수명']:
            name = foreigner.split(' ')
            names.extend(name)
            
        # 삭제
        p_remove_index = pitcher_final.loc[(pitcher_final['연도'] == YEAR) & (pitcher_final['팀명'] == TEAM) & (pitcher_final['선수명'].isin(names))].index
        p_remove_players = list(pitcher_final.loc[p_remove_index, '선수명'].unique())
        pitcher_final.drop(p_remove_index, inplace=True)
        
        h_remove_index = hitter_final.loc[(hitter_final['연도'] == YEAR) & (hitter_final['팀명'] == TEAM) & (hitter_final['선수명'].isin(names))].index
        h_remove_players = list(hitter_final.loc[h_remove_index, '선수명'].unique())
        hitter_final.drop(h_remove_index, inplace=True)
        
        if len(foreigners["선수명"].unique()) != len(p_remove_players + h_remove_players):
            print(f'전체삭제대상: {list(foreigners["선수명"].unique())}')
            print(f'{TEAM}({YEAR})의 투수: {p_remove_players}({len(p_remove_players)}명) 삭제 완료 == ')            
            print(f'{TEAM}({YEAR})의 타자: {h_remove_players}({len(h_remove_players)}명) 삭제 완료 == ')


2020 시작!!












2019 시작!!












2018 시작!!












2017 시작!!












2016 시작!!












2015 시작!!












2014 시작!!












2013 시작!!












2012 시작!!












2011 시작!!












2010 시작!!












2009 시작!!












2008 시작!!












2007 시작!!












2006 시작!!












2005 시작!!












2004 시작!!












2003 시작!!












2002 시작!!













In [7]:
pitcher_final = pitcher_final.sort_values(by = ['연도', '팀명','선수명']).reset_index(drop=True)
hitter_final = hitter_final.sort_values(by = ['연도', '팀명','선수명']).reset_index(drop=True)

pitcher_final.to_csv('선수데이터(전처리완료)/투수.csv', index=False, encoding='utf-8-sig')
hitter_final.to_csv('선수데이터(전처리완료)/타자.csv', index=False, encoding='utf-8-sig')